# Working with Verily Workbench groups

<table align="left">

  <td>
    <a href="https://github.com/verily-src/workbench-examples/blob/main/first_hour_on_vwb/working_with_groups.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>                                                                                             
</table>

## Overview

Access to resources and workspaces in Verily Workbench is handled by [Verily Workbench groups](https://support.workbench.verily.com/docs/guides/admin/user_groups/). This notebook provides a series of widgets that enable you to perform various tasks related to groups in Verily Workbench. All of the tasks accomplished by widgets in this notebook can also be accomplished via [Workbench CLI](https://support.workbench.verily.com/docs/guides/cli/cli_intro/#installing-and-running-the-workbench-cli) commands in your cloud environment's termnal. 

### Objective

Perform common workspace resource operations including:

- [View the Verily Workbench groups in which you're a member](#view_groups)
- [Create a new Verily Workbench group](#create_group)
- [Delete an existing Verily Workbench group](#delete_group)
- [Add and remove collaborators to and from your Verily Workbench group](#manage_membership)

#### How to run this notebook

Please run the Setup section before running any other sections in this worksapce. 

#### Costs

This notebook takes less than a minute to run, which will typically cost less than $0.01 of compute time on your cloud environment.

## Setup

Run the cell below to set up utilities for the widgets provided in this workspace.

In [37]:
from IPython.display import display, HTML
from typing import List
import csv
import json
import widget_utils as wu
import vwb_org_group_utils as ogu
import ipywidgets as widgets
import subprocess

## Legacy Verily Workbench groups
<a id='legacy_groups'></a>

### View legacy Workbench groups
On April 14, 2025, Workbench rolled out a change to the implementation of Workbench groups that allows users to create groups which are linked to their organization. Prior to this change, all groups were global in scope -- not linked to any organization. We refer to these global groups which predate this change as "legacy Workbench groups". To view any legacy Workbench groups in which you have a role, run the cell below.

In [40]:
result = subprocess.run(['wb','group','list','--format=JSON'],capture_output=True,text=True)
print(result.stderr) if not result.stdout else display(HTML(wu.list_legacy_groups(result.stdout)))

3p-sensors-workspace-write,['ADMIN']
aou-drc-gcp-pod,['MEMBER']
aou-initial-credit-stable-pod,['MEMBER']
aou-stable-rt,"['ADMIN', 'MEMBER']"
cdm-adhd-owner,['MEMBER']
cdm-demo-1-ws-owners,['MEMBER']
cdm-demo-1-ws-writers,['MEMBER']
cdm-sjogren-owner,['MEMBER']
cdm-workspace-write,['ADMIN']
colgate-ohs-vwb-dd-owners,['MEMBER']
colgate-ohs-vwb-demo-ws-owners,['ADMIN']


### View members of legacy Verily Workbench groups

In order to view the members of a legacy Verily Workbench group, you must be an admin of that group. To investigate the membership of your group, run the cell below to create a widget. Provide the group name in the widget input field and click the button to view a list of the members of the desired group.

In [39]:
class ListMembersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.output = widgets.Output()
        self.list_button = wu.StyledButton('List users in group','Click to list users in this Verily Workbench group.','list').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.list_button,self.output],layout=wu.vbox_layout)
        self.list_button.on_click(self.list_members)


    def list_members(self,b):
        self.output.clear_output()
        with self.output:
            wb_command = ["wb","group","list-users",f"--name={self.input_group_name.value}","--format=JSON"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            if not result.stdout:
                self.output.append_display_data(result.stderr)
            else:
                self.output.append_display_data(HTML(wu.list_legacy_group_members(result.stdout)))

# Create widget                
list_members_widget = ListMembersWidget()
display(list_members_widget.vb)

## Org-linked Verily Workbench groups
<a id='org_groups'></a>

### View org-linked Workbench groups
On April 14, 2025, Workbench rolled out a change to the implementation of Workbench groups that allows users to create groups which are linked to their organization. Prior to this change, all groups were global in scope -- not linked to any organization. We refer to these groups as "org-linked" groups. To view any org-linked Workbench groups in which you have a role, run the cell below.

In [ ]:
result = subprocess.run(["wb","group","list","--format=JSON"],capture_output=True,text=True)
display(HTML(wu.list_org_groups(result.stdout)))

### View members of org-linked Verily Workbench groups

In order to view the members of a org-linked Verily Workbench group, you must either be an admin of that group or an org admin. To investigate the membership of your group, run the cell below to create a widget. Provide the *group name* and the *org ID* in the widget input fields and click the button to view a list of the members of the desired group.

In [ ]:
class ListOrgLinkedMembersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>',"Org ID:").get()
        self.output = widgets.Output()
        self.list_button = wu.StyledButton('List users in group','Click to list users in this Verily Workbench group.','list').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.list_button,self.output],layout=wu.vbox_layout)
        self.list_button.on_click(self.list_members)
    
    def list_members(self,b):
        self.output.clear_output()
        with self.output:
            group_roles = ogu.get_org_linked_group_roles(self.input_org_id.value, self.input_group_name.value)
            html_roles = ogu.get_flat_roles_html(group_roles)
            self.output.append_display_data(HTML(html_roles))

# Create widget                
list_org_linked_members_widget = ListOrgLinkedMembersWidget()
display(list_org_linked_members_widget.vb)

## Create a new Verily Workbench group
<a id='create_group'></a>

Run the cell below to create a widget, then populate the widget's fields and click the button to create a new Verily Workbench group. 
The value provided for the `GROUP_NAME` should be unique, and reflect the purpose of the group (for example, for a group of Verily researchers working on the 1000 Genomes dataset, `verily-1000-genomes-researchers` would be appropriate).

You should see output resembling:

```
Workbench group created.
<GROUP_NAME>
  Email: <GROUP_NAME>@verily.com
  # Members: 1
  Current user's policies: ADMIN
```

In [ ]:
class CreateGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name & org ID for your Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Create group','Click to create a Verily Workbench group.','plus').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.button,self.output],layout=wu.vbox_layout)
        self.button.on_click(self.create_group)

    def create_group(self,b):
        with self.output:
            self.output.clear_output()
            wb_command = ["wb","group","create",f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}","--format=text"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
create_group_widget = CreateGroupWidget()
display(create_group_widget.vb)

## Delete a Verily Workbench group

<a id='delete_group'></a>

In order to delete a Verily Workbench group, you must be an admin of that group. Run the cell below to create a widget, then populate the input fields and click the button to delete the desired group.

**NOTE:** Once a Verily Workbench group has been deleted, this operation cannot be reversed. Please take care not to delete any groups currently in use.

Your output should resemble:
```
<GROUP_NAME>
  Email: <GROUP_NAME>@verily.com
  # Members: <NUMBER_OF_MEMBERS>
  Current user's policies: ADMIN
Workbench group successfully deleted.
```

In [ ]:
class DeleteGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Delete group','Click to delete a Verily Workbench group.','trash').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.button,self.output],layout=wu.vbox_layout)
        self.button.on_click(self.delete_group)

    def delete_group(self,b):
        with self.output:
            self.output.clear_output()
            wb_command = ["wb","group","delete",f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}","--quiet"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
delete_group_widget = DeleteGroupWidget()
display(delete_group_widget.vb)

## Manage Verily Workbench group membership
<a id='manage_membership'></a>

Managing group membership is often an ongoing task throught a project's lifecycle.
<br>The sub-sections below create widgets that empower you to manage group membership without having to use the Workbench CLI in the terminal.

**Note:** In order to manage the membership of a Verily Workbench group, you must have the 'ADMIN' policy in that group.

### Add a user to a Workbench group

1. Run the cell below to create a widget that adds a user to a Verily Workbench group.
1. In the widget, provide the group name and org ID of an existing Verily Workbench group and the email of the user you wish to add. 
1. Click the button to add the user to the group!

You should then see output resembling:
```
User added to Workbench group.
<EMAIL>: <POLICY>
```

In [ ]:
class AddUserWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_user_email = wu.TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.policy_drop_down = wu.DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Policy:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Add user','Click to add a user to the group.','user-plus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_org_id,
             self.input_user_email,
             self.policy_drop_down,
             self.button,
             self.output
            ],layout=wu.vbox_layout)
        self.button.on_click(self.add_user)

    def add_user(self,b):
        with self.output:
            wb_command = ["wb", "group", "add-user",f"--name={self.input_group_name.value}",f"--name={self.input_org_id.value}",f"--email={self.input_user_email.value}",f"--policy={self.policy_drop_down.value}"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
add_user_widget = AddUserWidget()
display(add_user_widget.vb)

### Remove a user from a Verily Workbench group

1. Run the cell below to create a widget.
1. In the widget, provide the group name for an existing Verily Workbench group and the email of the user you wish to remove. 
1. Click the button to remove the user from the group.

You should see output resembling:
```
User (<USER_EMAIL>) removed from policy (<POLICY>) in group (<GROUP_NAME>).
```

In [ ]:
class RemoveUserWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_user_email = wu.TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.policy_drop_down = wu.DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Policy:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Remove user','Click to remove a user from the group.','user-minus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_org_id,
             self.input_user_email,
             self.policy_drop_down,
             self.button,
             self.output
            ],layout=wu.vbox_layout)
        self.button.on_click(self.remove_user)

    def remove_user(self,b):
        with self.output:
            wb_command = ["wb", "group", "remove-user",f"--name={self.input_group_name.value}",f"--name={self.input_org_id.value}",f"--email={self.input_user_email.value}",f"--policy={self.policy_drop_down.value}"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
add_user_widget = RemoveUserWidget()
display(add_user_widget.vb)

### Add a batch of users to a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "WORKBENCH_USER_EMAIL" and "ROLE", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding membership policy (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to add a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User added to Workbench group.
<EMAIL>: <ROLE>
```

In [ ]:
class BatchAddUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:",).get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_file = wu.TextInputWidget('<FILE>',"CSV File:",).get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Add users','Click to add a batch of users to the group.','user-plus').get()
        self.button.on_click(self.batch_add_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.input_file,self.button,self.output],layout=wu.vbox_layout)

    def batch_add_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    wb_command = ["wb", "group", "role","grant", "user",f"--name={self.input_group_name.value}",f"--name={self.input_org_id.value}",f"--email={user['WORKBENCH_USER_EMAIL']}",f"--role={user['ROLE']}"]
                    result = subprocess.run(wb_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)



# Instantiate widget
batch_add_users_widget = BatchAddUsersWidget()
display(batch_add_users_widget.vb)

### Remove a batch of users from a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "WORKBENCH_USER_EMAIL" and "ROLE", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding membership policy (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to remove a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User (<USER_EMAIL>) removed from policy (<POLICY>) in group (<GROUP_NAME>).
```

In [ ]:
class BatchRemoveUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_file = wu.TextInputWidget('<FILE>',"CSV File:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Remove users','Click to remove a batch of users to the group.','user-minus').get()
        self.button.on_click(self.batch_remove_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.input_file,self.button,self.output],layout=wu.vbox_layout)

    def batch_remove_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    wb_command = ["wb", "group", "role", "revoke", "user", f"--name={self.input_group_name.value}",f"--name={self.input_group_name.value}",f"--email={user['WORKBENCH_USER_EMAIL']}",f"--policy={user['ROLE']}"]
                    result = subprocess.run(wb_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
batch_remove_users_widget = BatchRemoveUsersWidget()
display(batch_remove_users_widget.vb)

## Provenance

Generate information about this notebook environment and the packages installed.

In [ ]:
!date

Conda and pip installed packages:

In [ ]:
!conda env export

JupyterLab extensions:

In [ ]:
!jupyter labextension list

Number of cores:

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

Memory:

In [ ]:
!grep "^MemTotal:" /proc/meminfo

---
Copyright 2025 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd